```markdown
To run this Jupyter notebook, use Python 3.13 or later. The recommended environment is a virtual environment or a conda environment with the necessary packages installed, as specified int he requirements.txt file. Ensure that you have Jupyter Notebook installed to execute the cells interactively.
```

In [ ]:
%run reuse.ipynb

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create a dropdown widget for environment selection
env_dropdown = widgets.Dropdown(
    options=['dev', 'uat', 'prod'],
    value='dev',
    description='Environment:',
)

outputdirectory = widgets.Text("testdata")

# Display the widget
display(widgets.HBox([env_dropdown, outputdirectory]))




In [ ]:
selected_env = env_dropdown.value
outputdir = outputdirectory.value
healthData = HealthData(selected_env,outputdir) # type: ignore
healthData.generate_data()

In [ ]:
import pandas as pd

healthdatadf = pd.read_parquet(f"./{outputdir}/health_data_{selected_env}.parquet")

In [ ]:
####!pyspark --packages io.delta:delta-core_2.11:0.4.0
import pyspark
from pyspark.sql import SparkSession
from delta import *


builder = (
    SparkSession.builder.appName("DeltaTableCreation")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)


In [ ]:
filtered_healthdatadf = healthdatadf[healthdatadf['BloodPressure'] > 120]
print(filtered_healthdatadf)

# Use Spark SQL to filter the data and convert to Pandas DataFrame
spark.createDataFrame(healthdatadf).createOrReplaceTempView("sparkhealthdatadf")
filtered_healthdatadf_sql = spark.sql("SELECT * FROM sparkhealthdatadf WHERE BloodPressure > 120").toPandas()
print(filtered_healthdatadf_sql)

In [ ]:
# Create a Spark DataFrame from the healthdatadf DataFrame
sparkdf = spark.createDataFrame(healthdatadf)

# Write the DataFrame as a Delta table
sparkdf.write.format("delta").mode("overwrite").save(f"./{outputdir}/health_data_{selected_env}")

In [ ]:
# Read the Delta table
delta_table = spark.read.format("delta").load(f"./{outputdir}/health_data_{selected_env}")

filtered_data = delta_table.filter(delta_table.BloodPressure > 120)
filtered_data.show()

In [ ]:
# Create a temporary view
delta_table.createOrReplaceTempView(f"health_data")

# Use Spark SQL to filter the data
filtered_data_sql = spark.sql("SELECT * FROM health_data WHERE BloodPressure > 120")
filtered_data_sql.show()

In [ ]:
%pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
# Execute the SQL query and display the results
result = spark.sql("SELECT * FROM `parquet`.`./testdata/health_data_dev` WHERE BloodPressure > 120")
result.show()

In [ ]:
import os

# List all files in the subfolders of the testdata directory
file_list = [os.path.join(root, file) for root, dirs, files in os.walk(outputdir) for file in files if file.endswith('.csv') or file.endswith('.parquet')]

# Create a dropdown widget for file selection
file_dropdown = widgets.Dropdown(
    options=[file.replace('testdata/', '') for file in file_list],
    description='Files:',
)

# Display the widget
display(file_dropdown)


In [ ]:
file_extension= file_dropdown.value.split('.')[-1]

spark.sql(f"SELECT * FROM `{file_extension}`.`./{outputdir}/{file_dropdown.value}`").show()

In [ ]:
# List columns and data types
df.printSchema()

In [ ]:
# Describe the data
display(df.describe().show())
# Find the highest salary
highest_salary = df.agg({"Salary": "max"}).collect()[0][0]
print(f"Highest Salary: {highest_salary}: with Education {df.filter(df.Salary == highest_salary).select('Education').collect()[0][0]} Position: {df.filter(df.Salary == highest_salary).select('Position').collect()[0][0]}")

In [ ]:
print()

In [ ]:

database_name = file_dropdown.value.split('/')[0].replace('-', '')
table_name = file_dropdown.value.split('/')[1].split('.')[0].replace('_', '')

print(database_name)
print(table_name)


In [ ]:
sparkdf = spark.read.format(f"{file_extension}").option("header", "true").load(f"./{outputdir}/{file_dropdown.value}")

# Rename columns to remove invalid characters
for col in sparkdf.columns:
	new_col = col.replace(' ', '_').replace('(', '').replace(')', '').replace('\n', '').replace('\t', '').replace('=', '')
	sparkdf = sparkdf.withColumnRenamed(col, new_col)

# Create a database
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")

# Use the created database
spark.sql(f"USE {database_name}")

# Write the DataFrame as a Delta table in the created database
sparkdf.write.format("delta").mode("overwrite").saveAsTable(f"{table_name}")

# Verify that the table has been created and data has been loaded


In [ ]:
spark.sql("SHOW TABLES").show()
#spark.sql(f"SELECT * FROM {table_name}").show()
# Group by Location and show the count of each location
spark.sql(f"SELECT Location, COUNT(*) as count FROM {table_name} GROUP BY Location ORDER BY count DESC").show()


In [ ]:
# Count the number of people working from home
work_from_home_count = spark.sql(f"SELECT COUNT(*) as work_from_home FROM {table_name} WHERE Location like '%work from home%'")
work_from_home_count.show()
# Count unique locations
unique_locations_count = spark.sql(f"SELECT COUNT(DISTINCT Location) as unique_locations FROM {table_name}")
unique_locations_count.show()

In [ ]:
import matplotlib.pyplot as plt

# Convert the Spark DataFrame to a Pandas DataFrame
locations_df = sparkdf.select("Location").groupBy("Location").count().orderBy("count", ascending=False).toPandas()

# Fill None values in the 'Location' column with an empty string
locations_df['Location'] = locations_df['Location'].fillna('')

# Plot the locations
plt.figure(figsize=(10, 6))
plt.bar(locations_df['Location'], locations_df['count'])
plt.xlabel('Location')
plt.ylabel('Count')
plt.title('Count of Locations')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Execute the DESCRIBE EXTENDED query
describe_df = spark.sql(f"DESCRIBE EXTENDED {table_name}")

# Convert to Pandas DataFrame for better display
describe_pd_df = describe_df.toPandas()

# Display the DataFrame
display(describe_pd_df)

location_data_type = describe_pd_df[describe_pd_df['col_name'] == 'Location']['data_type'].values[1]
print(location_data_type)